# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

**Library Imports**
The first thing we need to do is import the open-source software libraries that we'll be using

In [4]:
!pip install xlsxwriter

     |████████████████████████████████| 153kB 2.9MB/s 


In [5]:

import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

In [6]:

stocks = pd.read_csv('/content/sp_500_stocks.csv')
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

**Making Our First API Call**
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [7]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 95713577,
 'calculationPrice': 'close',
 'change': -1.97,
 'changePercent': -0.01537,
 'close': 126.15,
 'closeSource': 'fiflcioa',
 'closeTime': 1644260133339,
 'companyName': 'Apple Inc',
 'delayedPrice': 126.76,
 'delayedPriceTime': 1685891767897,
 'extendedChange': -0.13,
 'extendedChangePercent': -0.00097,
 'extendedPrice': 126.14,
 'extendedPriceTime': 1665809408245,
 'high': 131,
 'highSource': '1yimueedn5tecea d i rpl',
 'highTime': 1683612627052,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 131.41,
 'iexCloseTime': 1685863451972,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 128.61,
 'iexOpenTime': 1700268394100,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'isUSMarketOpen': False,
 'lastTradeTime': 1651647412424,
 'latestPrice': 127.78,
 'latestSource': 'Close',
 'latestTime': 'May 21, 2021',
 'latestUpdate': 1690487250076,
 'latestVolume': 82355663,
 'low':

In [8]:
pe_ratio = data['peRatio']
pe_ratio

28.85

Executing A Batch API Call & Building Our DataFrame
Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [10]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [11]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,137.30,52.27,N/A
1,AAL,23.09,-1.55,N/A
2,AAP,200.98,27.6,N/A
3,AAPL,131.15,29,N/A
4,ABBV,120.79,41.32,N/A
...,...,...,...,...
500,YUM,123.98,33.35,N/A
501,ZBH,174.32,63.14,N/A
502,ZBRA,506.63,41.44,N/A
503,ZION,60.30,7.26,N/A


# Removing Glamour Stocks
The opposite of a "value stock" is a "glamour stock".

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [12]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)


# Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy.

To do this, we will use the portfolio_input function that we created in our momentum project.

I have included this function below.

In [13]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

In [14]:
portfolio_input()

Enter the value of your portfolio:5000


In [15]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,607.97,4.48,0
1,CINF,124.62,6.51,0
2,ZION,60.30,7.26,1
3,AFL,56.09,7.5,1
4,UNM,30.68,7.99,3
5,COO,402.08,8.75,0
6,HOLX,64.36,8.77,1
7,KIM,20.61,9.06,4
8,PGR,101.01,9.17,0
9,CE,165.17,9.2,0


**Building a Better (and More Realistic) Value Strategy**
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a composite basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

Price-to-earnings ratio
Price-to-book ratio
Price-to-sales ratio
Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
Enterprise Value divided by Gross Profit (EV/GP)
Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [16]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

In [17]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

Dealing With Missing Data in Our DataFrame
Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using.

You can use pandas' isnull method to identify missing data:

In [18]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,262.330,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,289.580,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.92,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,29.800,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.660,N/A,14.84,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,64.110,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,60.140,N/A,32.12,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,37.920,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,192.110,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2377.700,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

Drop missing data from the data set (pandas' dropna method is useful here)
Replace missing data with a new value (pandas' fillna method is useful here)
In this tutorial, we will replace missing data with the average non-NaN data point from that column.

Here is the code to do this:

In [19]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

In [21]:

rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


# Calculating Value Percentiles
We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

Price-to-earnings ratio
Price-to-book ratio
Price-to-sales ratio
EV/EBITDA
EV/GP
Here's how we'll do this:

In [22]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.831683
1      0.130693
2       0.49703
3      0.526733
4      0.760396
         ...   
500     0.60198
501    0.881188
502    0.762376
503    0.142574
504    0.817822
Name: PE Percentile, Length: 505, dtype: object
0       0.780198
1      0.0871287
2       0.505941
3       0.956436
4       0.879208
         ...    
500    0.0851485
501     0.392079
502     0.833663
503     0.135644
504     0.930693
Name: PB Percentile, Length: 505, dtype: object
0      0.790099
1      0.110891
2      0.125743
3      0.756436
4      0.560396
         ...   
500    0.740594
501    0.625743
502    0.720792
503    0.381188
504    0.912871
Name: PS Percentile, Length: 505, dtype: object
0       0.877228
1       0.039604
2       0.352475
3       0.647525
4       0.322772
         ...    
500     0.708911
501     0.809901
502     0.833663
503    0.0574257
504     0.881188
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0      0.786139
1      0.128713
2      0.124752
3      0.873267
4      0.49

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,135.20,N/A,52.41,0.831683,8.58,0.780198,7.43,0.790099,32.774608,0.877228,13.870427,0.786139,N/A
1,AAL,23.23,N/A,-1.55,0.130693,-1.87,0.0871287,1.17,0.110891,-3.668875,0.039604,3.067451,0.128713,N/A
2,AAP,200.02,N/A,27.90,0.49703,3.77,0.505941,1.28,0.125743,13.116610,0.352475,2.953390,0.124752,N/A
3,AAPL,129.93,N/A,28.80,0.526733,31.15,0.956436,6.67,0.756436,21.518119,0.647525,16.751806,0.873267,N/A
4,ABBV,120.65,N/A,41.04,0.760396,15.09,0.879208,4.24,0.560396,12.658616,0.322772,8.466228,0.491089,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,125.39,N/A,32.60,0.60198,-4.55,0.0851485,6.21,0.740594,24.373861,0.708911,11.125487,0.685149,N/A
501,ZBH,167.60,N/A,62.42,0.881188,2.80,0.392079,5.08,0.625743,28.688350,0.809901,8.303178,0.477228,N/A
502,ZBRA,494.36,N/A,41.58,0.762376,11.19,0.833663,5.84,0.720792,31.586476,0.833663,12.434649,0.744554,N/A
503,ZION,59.60,N/A,7.27,0.142574,1.30,0.135644,2.82,0.381188,5.743252,0.0574257,2.798972,0.112871,N/A


# Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [23]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,135.20,N/A,52.41,0.831683,8.58,0.780198,7.43,0.790099,32.774608,0.877228,13.870427,0.786139,0.813069
1,AAL,23.23,N/A,-1.55,0.130693,-1.87,0.0871287,1.17,0.110891,-3.668875,0.039604,3.067451,0.128713,0.0994059
2,AAP,200.02,N/A,27.90,0.49703,3.77,0.505941,1.28,0.125743,13.116610,0.352475,2.953390,0.124752,0.321188
3,AAPL,129.93,N/A,28.80,0.526733,31.15,0.956436,6.67,0.756436,21.518119,0.647525,16.751806,0.873267,0.752079
4,ABBV,120.65,N/A,41.04,0.760396,15.09,0.879208,4.24,0.560396,12.658616,0.322772,8.466228,0.491089,0.602772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,125.39,N/A,32.60,0.60198,-4.55,0.0851485,6.21,0.740594,24.373861,0.708911,11.125487,0.685149,0.564356
501,ZBH,167.60,N/A,62.42,0.881188,2.80,0.392079,5.08,0.625743,28.688350,0.809901,8.303178,0.477228,0.637228
502,ZBRA,494.36,N/A,41.58,0.762376,11.19,0.833663,5.84,0.720792,31.586476,0.833663,12.434649,0.744554,0.77901
503,ZION,59.60,N/A,7.27,0.142574,1.30,0.135644,2.82,0.381188,5.743252,0.0574257,2.798972,0.112871,0.165941


# Selecting the 50 Best Value Stocks

In [24]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

# Calculating the Number of Shares to Buy
We'll use the portfolio_input function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [25]:
portfolio_input()

Enter the value of your portfolio:5000


In [26]:

position_size = float(portfolio_size) / len(rv_dataframe.index)
print("position_size: ",position_size)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

position_size:  100.0


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,30.65,3,8.07,0.144554,0.5927,0.0891089,0.4626,0.0217822,3.881308,0.049505,0.459578,0.00990099,0.0629703
1,MCK,209.72,0,-7.41,0.10495,-1601.0300,0.0039604,0.1403,0.00792079,7.770324,0.0930693,2.750750,0.106931,0.0633663
2,AIV,7.15,13,-78.00,0.0277228,2.0500,0.290099,-31.8600,0.0019802,-44.548021,0.00792079,-54.755525,0.00594059,0.0667327
3,AIG,52.46,1,-12.09,0.0910891,0.7260,0.0930693,1.0300,0.0871287,6.616843,0.0673267,1.041548,0.0277228,0.0732673
4,L,58.98,1,-423.50,0.0039604,0.9153,0.10297,1.0900,0.0970297,8.846315,0.142574,1.082722,0.029703,0.0752475
5,HFC,34.72,2,-37.39,0.0514851,1.0900,0.112871,0.5069,0.0237624,8.452493,0.116832,2.863096,0.118812,0.0847525
6,ABC,122.41,0,-6.68,0.114851,-90.7100,0.00792079,0.1292,0.00594059,8.884324,0.146535,4.373272,0.207921,0.0966337
7,FTI,8.60,11,11.12,0.182178,1.0100,0.108911,0.3238,0.0118812,3.683596,0.0475248,3.429546,0.144554,0.0990099
8,AAL,23.23,4,-1.55,0.130693,-1.8700,0.0871287,1.1700,0.110891,-3.668875,0.039604,3.067451,0.128713,0.0994059
9,BA,244.47,0,-11.89,0.0930693,-7.7000,0.0772277,2.4500,0.341584,-19.679444,0.0217822,-41.518174,0.00792079,0.108317


# Formatting Our Excel Output
We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [27]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [28]:
writer.save()